In [1]:
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from molvs import standardize_smiles
from copy import deepcopy
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.model_selection import permutation_test_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from sklearn.metrics import pairwise_distances
import joblib
import pickle
from numpy import savetxt
from padelpy import from_sdf
import shap
from tqdm.notebook import tqdm

In [2]:
def convert_smi_to_canon_smi(smi):

    try:
        canon_smi = Chem.MolToSmiles(Chem.MolFromSmiles(smi),isomericSmiles = False)
    except:
        canon_smi='wrong_smiles'
    return canon_smi

In [3]:
def standart(smi):
    global m
    if smi!='wrong_smiles':
        try:
            smiles=standardize_smiles(smi)
            m = Chem.MolFromSmiles(smi)
        except:
            smi='error kekule'
    else:
        m = 'check the smiles'
    return m

In [4]:
df_ws=pd.read_csv('rat_oral_LD50_WS.csv')
df_ws

,CAS_Number,SMILES,pLD50,"LD50,mg/kg"
0,626-48-2,Cc1cc(=O)[nH]c(=O)[nH]1,0.291207,64463.0000
1,27849-94-1,CC(CCc1ccc2c(c1)OCO2)NN,0.440660,75449.0000
2,110-54-3,CCCCCC,0.537460,24980.0000
3,1330-92-3,CCCCC(CC)COC(=O)C1=C(C(=O)OCC(CC)CCCC)CCCC1,0.539250,113917.0000
4,57-55-6,CC(O)CO,0.580330,19989.0000
...,...,...,...,...
7707,56073-10-0,O=c1oc2ccccc2c(O)c1C1CC(c2ccc(-c3ccc(Br)cc3)cc...,6.514700,0.1596
7708,130209-82-4,CC(C)OC(=O)CCC/C=C/CC1C(O)CC(O)C1CCC(O)CCc1ccccc1,6.937100,0.0500
7709,83805-11-2,C=C1/C(=C\C=C2/CCCC3(C)C2CCC3C(C)CCCC(O)(C(F)(...,7.099700,0.0417
7710,1746-01-6,Clc1cc2c(cc1Cl)Oc1cc(Cl)c(Cl)cc1O2,7.206800,0.0199


In [5]:
df_ws1 = deepcopy(df_ws)
df_ws1["SMILES"] = df_ws1.apply(lambda x: convert_smi_to_canon_smi(x.SMILES), axis=1)
df_ws1

,CAS_Number,SMILES,pLD50,"LD50,mg/kg"
0,626-48-2,Cc1cc(=O)[nH]c(=O)[nH]1,0.291207,64463.0000
1,27849-94-1,CC(CCc1ccc2c(c1)OCO2)NN,0.440660,75449.0000
2,110-54-3,CCCCCC,0.537460,24980.0000
3,1330-92-3,CCCCC(CC)COC(=O)C1=C(C(=O)OCC(CC)CCCC)CCCC1,0.539250,113917.0000
4,57-55-6,CC(O)CO,0.580330,19989.0000
...,...,...,...,...
7707,56073-10-0,O=c1oc2ccccc2c(O)c1C1CC(c2ccc(-c3ccc(Br)cc3)cc...,6.514700,0.1596
7708,130209-82-4,CC(C)OC(=O)CCCC=CCC1C(O)CC(O)C1CCC(O)CCc1ccccc1,6.937100,0.0500
7709,83805-11-2,C=C1C(=CC=C2CCCC3(C)C2CCC3C(C)CCCC(O)(C(F)(F)F...,7.099700,0.0417
7710,1746-01-6,Clc1cc2c(cc1Cl)Oc1cc(Cl)c(Cl)cc1O2,7.206800,0.0199


In [6]:
print('Original data: ', len(df_ws), 'molecules')
print('Failed data: ', len(df_ws1[df_ws1['SMILES']=='wrong_smiles']), 'molecules')

Original data:  7712 molecules
Failed data:  0 molecules


In [7]:
index=df_ws1.index[df_ws1['SMILES']=='wrong_smiles'].tolist()
wrong_smiles=df_ws.iloc[index]
wrong_smiles=wrong_smiles.SMILES
number=[x+1 for x in index]
bad_molecules = pd.DataFrame({'No. failed smiles in original set': number, 'SMILES of wrong structure: ': wrong_smiles}, index=None)
bad_molecules = bad_molecules.set_index('No. failed smiles in original set')
bad_molecules

,SMILES of wrong structure:
No. failed smiles in original set,


2.2  Standardization for work set

In [8]:
df_ws1["Molecule"] = df_ws1.apply(lambda x: standart(x.SMILES), axis=1)
moldf_ws=df_ws1[df_ws1['SMILES']!='wrong_smiles']
print('Kept data: ', len(moldf_ws), 'molecules')

Kept data:  7712 molecules


In [9]:
moldf_ws

,CAS_Number,SMILES,pLD50,"LD50,mg/kg",Molecule
0,626-48-2,Cc1cc(=O)[nH]c(=O)[nH]1,0.291207,64463.0000,<rdkit.Chem.rdchem.Mol object at 0x0000014A970...
1,27849-94-1,CC(CCc1ccc2c(c1)OCO2)NN,0.440660,75449.0000,<rdkit.Chem.rdchem.Mol object at 0x0000014A970...
2,110-54-3,CCCCCC,0.537460,24980.0000,<rdkit.Chem.rdchem.Mol object at 0x0000014A970...
3,1330-92-3,CCCCC(CC)COC(=O)C1=C(C(=O)OCC(CC)CCCC)CCCC1,0.539250,113917.0000,<rdkit.Chem.rdchem.Mol object at 0x0000014A970...
4,57-55-6,CC(O)CO,0.580330,19989.0000,<rdkit.Chem.rdchem.Mol object at 0x0000014A972...
...,...,...,...,...,...
7707,56073-10-0,O=c1oc2ccccc2c(O)c1C1CC(c2ccc(-c3ccc(Br)cc3)cc...,6.514700,0.1596,<rdkit.Chem.rdchem.Mol object at 0x0000014A982...
7708,130209-82-4,CC(C)OC(=O)CCCC=CCC1C(O)CC(O)C1CCC(O)CCc1ccccc1,6.937100,0.0500,<rdkit.Chem.rdchem.Mol object at 0x0000014A982...
7709,83805-11-2,C=C1C(=CC=C2CCCC3(C)C2CCC3C(C)CCCC(O)(C(F)(F)F...,7.099700,0.0417,<rdkit.Chem.rdchem.Mol object at 0x0000014A982...
7710,1746-01-6,Clc1cc2c(cc1Cl)Oc1cc(Cl)c(Cl)cc1O2,7.206800,0.0199,<rdkit.Chem.rdchem.Mol object at 0x0000014A982...


In [10]:
y_tr=moldf_ws.pLD50
y_tr

0       0.291207
1       0.440660
2       0.537460
3       0.539250
4       0.580330
          ...   
7707    6.514700
7708    6.937100
7709    7.099700
7710    7.206800
7711    7.602600
Name: pLD50, Length: 7712, dtype: float64

In [11]:
df = pd.DataFrame(moldf_ws, columns=["SMILES"])
df

,SMILES
0,Cc1cc(=O)[nH]c(=O)[nH]1
1,CC(CCc1ccc2c(c1)OCO2)NN
2,CCCCCC
3,CCCCC(CC)COC(=O)C1=C(C(=O)OCC(CC)CCCC)CCCC1
4,CC(O)CO
...,...
7707,O=c1oc2ccccc2c(O)c1C1CC(c2ccc(-c3ccc(Br)cc3)cc...
7708,CC(C)OC(=O)CCCC=CCC1C(O)CC(O)C1CCC(O)CCc1ccccc1
7709,C=C1C(=CC=C2CCCC3(C)C2CCC3C(C)CCCC(O)(C(F)(F)F...
7710,Clc1cc2c(cc1Cl)Oc1cc(Cl)c(Cl)cc1O2


In [12]:
df.to_csv('datasets/molecule_ws.smi', sep=',', index=False, header=False)

2.3  Load data and curation test set

In [13]:
df_ts=pd.read_csv('rat_oral_LD50_TS.csv')
df_ts

,CAS_Number,SMILES,pLD50,"LD50,mg/kg"
0,7782-40-3,C,0.017765,15388.8000
1,2842-38-8,OCCNC1CCCCC1,0.572840,38274.0000
2,66257-53-2,NC(=O)C(=O)O,0.624490,21133.0000
3,2173-56-0,CCCCCOC(=O)CCCC,0.686960,35395.0000
4,4726-93-6,O=C1CCCCC(=O)N1,0.750180,22586.0000
...,...,...,...,...
1924,3385-03-3,CC1(C)OC2CC3C4CC(F)C5=CC(=O)C=CC5(C)C4C(O)CC3(...,5.939000,0.4997
1925,2338-29-6,FC(F)(F)c1nc2c(Cl)c(Cl)c(Cl)c(Cl)c2[nH]1,6.121300,0.2435
1926,128606-48-4,CCOP(=S)(OCC)O/C(C)=C/C(=O)OC,6.282400,0.1399
1927,50585-41-6,Brc1cc2c(cc1Br)Oc1cc(Br)c(Br)cc1O2,6.698800,0.0992


In [14]:
df_ts1 = deepcopy(df_ts)
df_ts1["SMILES"] = df_ts1.apply(lambda x: convert_smi_to_canon_smi(x.SMILES), axis=1)
df_ts1

,CAS_Number,SMILES,pLD50,"LD50,mg/kg"
0,7782-40-3,C,0.017765,15388.8000
1,2842-38-8,OCCNC1CCCCC1,0.572840,38274.0000
2,66257-53-2,NC(=O)C(=O)O,0.624490,21133.0000
3,2173-56-0,CCCCCOC(=O)CCCC,0.686960,35395.0000
4,4726-93-6,O=C1CCCCC(=O)N1,0.750180,22586.0000
...,...,...,...,...
1924,3385-03-3,CC1(C)OC2CC3C4CC(F)C5=CC(=O)C=CC5(C)C4C(O)CC3(...,5.939000,0.4997
1925,2338-29-6,FC(F)(F)c1nc2c(Cl)c(Cl)c(Cl)c(Cl)c2[nH]1,6.121300,0.2435
1926,128606-48-4,CCOP(=S)(OCC)OC(C)=CC(=O)OC,6.282400,0.1399
1927,50585-41-6,Brc1cc2c(cc1Br)Oc1cc(Br)c(Br)cc1O2,6.698800,0.0992


In [15]:
print('Original data: ', len(df_ts), 'molecules')
print('Failed data: ', len(df_ts1[df_ts1['SMILES']=='wrong_smiles']), 'molecules')

Original data:  1929 molecules
Failed data:  0 molecules


In [16]:
index=df_ts1.index[df_ts1['SMILES']=='wrong_smiles'].tolist()
wrong_smiles=df_ts.iloc[index]
wrong_smiles=wrong_smiles.SMILES
number=[x+1 for x in index]
bad_molecules = pd.DataFrame({'No. failed smiles in original set': number, 'SMILES of wrong structure: ': wrong_smiles}, index=None)
bad_molecules = bad_molecules.set_index('No. failed smiles in original set')
bad_molecules

,SMILES of wrong structure:
No. failed smiles in original set,


2.4  Standardization for test set

In [17]:
df_ts1["Molecule"] = df_ts1.apply(lambda x: standart(x.SMILES), axis=1)
moldf_ts=df_ts1[df_ts1['SMILES']!='wrong_smiles']
print('Kept data: ', len(moldf_ts), 'molecules')

Kept data:  1929 molecules


In [18]:
moldf_ts

,CAS_Number,SMILES,pLD50,"LD50,mg/kg",Molecule
0,7782-40-3,C,0.017765,15388.8000,<rdkit.Chem.rdchem.Mol object at 0x0000014A971...
1,2842-38-8,OCCNC1CCCCC1,0.572840,38274.0000,<rdkit.Chem.rdchem.Mol object at 0x0000014A971...
2,66257-53-2,NC(=O)C(=O)O,0.624490,21133.0000,<rdkit.Chem.rdchem.Mol object at 0x0000014A971...
3,2173-56-0,CCCCCOC(=O)CCCC,0.686960,35395.0000,<rdkit.Chem.rdchem.Mol object at 0x0000014A971...
4,4726-93-6,O=C1CCCCC(=O)N1,0.750180,22586.0000,<rdkit.Chem.rdchem.Mol object at 0x0000014A971...
...,...,...,...,...,...
1924,3385-03-3,CC1(C)OC2CC3C4CC(F)C5=CC(=O)C=CC5(C)C4C(O)CC3(...,5.939000,0.4997,<rdkit.Chem.rdchem.Mol object at 0x0000014A970...
1925,2338-29-6,FC(F)(F)c1nc2c(Cl)c(Cl)c(Cl)c(Cl)c2[nH]1,6.121300,0.2435,<rdkit.Chem.rdchem.Mol object at 0x0000014A970...
1926,128606-48-4,CCOP(=S)(OCC)OC(C)=CC(=O)OC,6.282400,0.1399,<rdkit.Chem.rdchem.Mol object at 0x0000014A970...
1927,50585-41-6,Brc1cc2c(cc1Br)Oc1cc(Br)c(Br)cc1O2,6.698800,0.0992,<rdkit.Chem.rdchem.Mol object at 0x0000014A970...


In [19]:
y_ts=moldf_ts.pLD50
y_ts

0       0.017765
1       0.572840
2       0.624490
3       0.686960
4       0.750180
          ...   
1924    5.939000
1925    6.121300
1926    6.282400
1927    6.698800
1928    9.541100
Name: pLD50, Length: 1929, dtype: float64

In [24]:
df = pd.DataFrame(moldf_ts, columns=["SMILES"])
df.to_csv('datasets/molecule_ts.smi', sep=',', index=False, header=False)

# 6.Descriptor calculation for work set

## AtomPairs2D FPs

In [163]:
from padelpy import padeldescriptor

fingerprint = 'AtomPairs2D'

fingerprint_output_file = ''.join([fingerprint,'.csv']) #Substructure.csv
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='datasets/molecule_ws.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

In [32]:
descriptors_AtomPairs2D = pd.read_csv('AtomPairs2D.csv')

In [33]:
descriptors_AtomPairs2D

,Name,AD2D1,AD2D2,AD2D3,AD2D4,AD2D5,AD2D6,AD2D7,AD2D8,AD2D9,...,AD2D771,AD2D772,AD2D773,AD2D774,AD2D775,AD2D776,AD2D777,AD2D778,AD2D779,AD2D780
0,AUTOGEN_molecule_ws_1,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,AUTOGEN_molecule_ws_2,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,AUTOGEN_molecule_ws_3,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,AUTOGEN_molecule_ws_4,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,AUTOGEN_molecule_ws_5,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7707,AUTOGEN_molecule_ws_7708,1,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
7708,AUTOGEN_molecule_ws_7709,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7709,AUTOGEN_molecule_ws_7710,1,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7710,AUTOGEN_molecule_ws_7711,1,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
x_tr = descriptors_AtomPairs2D.drop('Name', axis=1)

In [35]:
x_tr = np.array(x_tr, dtype=np.float32)
y_tr = np.array(y_tr, dtype=np.float32)

In [124]:
savetxt('Models/Padels/x_tr_AtomPairs2D.csv', y_tr, delimiter=',')

In [36]:
x_tr.shape

(7712, 780)

# 7.Descriptor calculation for test set

In [179]:
from padelpy import padeldescriptor

fingerprint = 'AtomPairs2D'

fingerprint_output_file = ''.join([fingerprint,'_ts.csv']) #AtomPairs2D.csv
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='datasets/molecule_ts.smi', 
                d_file=fingerprint_output_file, #'AtomPairs2D.csv'
                #descriptortypes='AtomPairs2D.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=False,
                fingerprints=True)

In [24]:
descriptors_AtomPairs2D_ts = pd.read_csv('AtomPairs2D_ts.csv')

In [25]:
x_ts = descriptors_AtomPairs2D_ts.drop('Name', axis=1)

In [26]:
x_ts.shape

(1929, 780)

# load the models from disk

In [29]:
best_svm = pickle.load(open('models/Padels/Toxicity_SVM_AtomPairs2D.pkl', 'rb'))

In [28]:
best_gbr = pickle.load(open('models/Padels/GBR_AtomPairs2D.pkl', 'rb'))

In [30]:
best_MLPR = pickle.load(open('models/Padels/Toxicity_MLPR_AtomPairs2D.pkl', 'rb'))

# Prediction 

In [37]:
y_pred_ws_GBR = best_gbr.predict(x_tr)

In [38]:
y_pred_ws_svm = best_svm.predict(x_tr)

In [39]:
y_pred_con=(y_pred_ws_GBR+y_pred_ws_svm)/2

In [40]:
R2_WS = round(r2_score(y_tr, y_pred_con), 2)
R2_WS

0.78

In [41]:
RMSE_WS=round(np.sqrt(mean_absolute_error(y_tr, y_pred_con)), 2)
RMSE_WS

0.54

In [42]:
seed = 42
cv=KFold(n_splits=5, random_state=seed, shuffle=True)

In [43]:
y_pred_CV_svm = cross_val_predict(best_svm, x_tr, y_tr, cv=cv)

In [44]:
y_pred_CV_gbr = cross_val_predict(best_gbr, x_tr, y_tr, cv=cv)

In [45]:
y_pred_CV_MLPR = cross_val_predict(best_MLPR, x_tr, y_tr, cv=cv)

# For all models

In [46]:
y_pred_con=(y_pred_CV_svm+y_pred_CV_gbr+y_pred_CV_MLPR)/3

In [47]:
Q2_CV = round(r2_score(y_tr, y_pred_con), 2)
Q2_CV

0.49

In [48]:
RMSE_CV=round(np.sqrt(mean_absolute_error(y_tr, y_pred_con)),2)
RMSE_CV

0.68

# two models: svm+gbr

In [49]:
y_pred_con_without_MLPR=(y_pred_CV_svm+y_pred_CV_gbr)/2

In [50]:
Q2_CV = round(r2_score(y_tr, y_pred_con_without_MLPR), 2)
Q2_CV

0.49

In [51]:
RMSE_CV=round(np.sqrt(mean_absolute_error(y_tr, y_pred_con_without_MLPR)),2)
RMSE_CV

0.68

# Prediction for test set's molecules

In [79]:
x_ts = np.array(x_ts, dtype=np.float32)
y_ts = np.array(y_ts, dtype=np.float32)

In [80]:
y_pred_svm = best_svm.predict(x_ts)

In [81]:
y_pred_gbr = best_gbr.predict(x_ts)

In [82]:
y_pred_con=(y_pred_svm+y_pred_gbr)/2

In [83]:
Q2_TS = round(r2_score(y_ts, y_pred_con), 2)
Q2_TS

0.5

In [84]:
RMSE_TS=round(np.sqrt(mean_absolute_error(y_ts, y_pred_con)), 2)
RMSE_TS

0.68

# Estimating applicability domain. Method - Euclidian distances, K=1

In [85]:
neighbors_k= pairwise_distances(x_tr, n_jobs=-1)
neighbors_k.sort(0)

In [86]:
df_tr=pd.DataFrame(neighbors_k)
df_tr

,0,1,2,3,4,5,6,7,8,9,...,7702,7703,7704,7705,7706,7707,7708,7709,7710,7711
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1.000000,2.449490,0.000000,0.000000,0.000000,0.000000,2.645751,0.000000,0.000000,1.000000,...,1.732051,2.236068,2.000000,3.316625,1.000000,1.732051,0.000000,3.316625,2.828427,1.732051
2,1.000000,2.449490,0.000000,0.000000,0.000000,0.000000,2.645751,0.000000,0.000000,1.000000,...,2.449490,2.449490,2.236068,3.464102,2.645751,1.732051,0.000000,3.316625,3.316625,2.000000
3,1.414214,2.449490,0.000000,0.000000,1.000000,1.732051,2.645751,0.000000,0.000000,2.645751,...,2.449490,3.000000,2.449490,3.605551,3.000000,2.000000,1.414214,3.464102,3.741657,2.236068
4,1.732051,2.645751,0.000000,0.000000,1.000000,1.732051,2.828427,0.000000,0.000000,2.828427,...,2.645751,3.162278,2.449490,3.741657,3.162278,2.000000,1.414214,3.464102,3.741657,2.236068
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7707,10.246951,9.848858,10.677078,10.049875,10.723805,10.392304,10.198039,10.000000,10.049875,10.954452,...,10.535654,10.344080,10.049875,9.797959,10.440307,10.049875,10.148891,9.746795,10.198039,9.899495
7708,10.440307,9.899495,10.723805,10.198039,10.770329,10.440307,10.535654,10.148891,10.099504,11.000000,...,10.583005,10.677078,10.049875,9.848858,10.677078,10.295630,10.344080,9.848858,10.198039,10.000000
7709,10.862781,10.344080,10.862781,10.630146,11.000000,10.816654,10.630146,10.583005,10.535654,11.135529,...,10.954452,10.770329,10.630146,9.899495,10.862781,10.723805,10.770329,10.049875,10.862781,10.630146
7710,11.135529,10.535654,11.224972,10.677078,11.357817,11.045361,10.816654,10.770329,10.630146,11.313708,...,11.313708,10.954452,10.677078,9.899495,11.135529,10.770329,10.862781,10.049875,11.224972,10.630146


In [87]:
similarity= neighbors_k

In [88]:
Dmean=np.mean(similarity[1,:])

In [89]:
round(Dmean, 2)

1.36

In [90]:
std=np.std(similarity[1,:])

In [91]:
round(std, 2)

1.35

In [92]:
model_AD_limit=Dmean+std*0.5
print(np.round(model_AD_limit, 2))

2.03


In [93]:
neighbors_k_ts= pairwise_distances(x_tr,Y=x_ts, n_jobs=-1)
neighbors_k_ts.sort(0)

In [94]:
x_ts_AD=pd.DataFrame(neighbors_k_ts)
x_ts_AD

,0,1,2,3,4,5,6,7,8,9,...,1919,1920,1921,1922,1923,1924,1925,1926,1927,1928
0,1.000000,0.000000,1.732051,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,3.464102,2.449490,2.449490,0.000000,2.236068,0.000000,1.732051,3.741657,1.000000
1,1.000000,0.000000,2.000000,0.000000,1.414214,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,4.358899,3.162278,2.645751,1.414214,2.449490,1.732051,2.645751,3.872983,1.000000
2,1.000000,0.000000,2.000000,0.000000,1.414214,1.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,4.582576,3.316625,2.645751,1.414214,2.828427,2.449490,3.000000,4.242640,1.000000
3,1.414214,0.000000,2.236068,0.000000,1.414214,1.000000,0.000000,1.000000,1.000000,0.000000,...,1.000000,4.582576,3.605551,2.828427,1.732051,3.000000,2.449490,3.000000,4.358899,1.000000
4,1.414214,1.000000,2.236068,0.000000,1.414214,1.000000,0.000000,1.000000,1.000000,0.000000,...,1.000000,4.582576,3.605551,3.000000,2.236068,3.162278,2.449490,3.000000,4.472136,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7707,10.908712,10.148891,10.630146,10.148891,10.392304,10.440307,10.099504,10.770329,10.630146,10.099504,...,10.148891,10.000000,11.269427,10.000000,10.770329,9.848858,10.954452,10.488089,10.862781,10.295630
7708,10.954452,10.198039,10.770329,10.198039,10.488089,10.488089,10.148891,10.816654,10.770329,10.148891,...,10.246951,10.099504,11.313708,10.295630,10.816654,9.848858,10.954452,10.816654,10.862781,10.392304
7709,11.090536,10.583005,11.045361,10.630146,10.816654,10.723805,10.535654,10.954452,11.000000,10.583005,...,10.770329,10.148891,11.313708,10.488089,11.045361,9.949874,11.000000,10.908712,10.862781,10.908712
7710,11.445523,10.862781,11.313708,10.723805,11.090536,11.090536,10.723805,11.313708,11.357817,10.677078,...,10.862781,10.198039,11.357817,10.535654,11.090536,10.000000,11.045361,11.090536,11.224972,10.954452


In [95]:
similarity_ts= neighbors_k_ts
cpd_AD=similarity_ts[0,:]
cpd_value = np.round(cpd_AD, 3)
print(cpd_value)

[1.    0.    1.732 ... 1.732 3.742 1.   ]


In [96]:
cpd_AD = np.where(cpd_value <= model_AD_limit, True, False)
print(cpd_AD)

[ True  True  True ...  True False  True]


In [97]:
print("Coverage = ", sum(cpd_AD) / len(cpd_AD))

Coverage =  0.7257646448937273


In [98]:
print("Indices of substances included in AD = ", np.where(cpd_AD != 0)[0])

Indices of substances included in AD =  [   0    1    2 ... 1925 1926 1928]


In [99]:
out_Ad=list(np.where(cpd_AD == 0)[0])

# Prediction only for molecules included in  AD

In [100]:
y_pred_con_ad=list(y_pred_con)

In [101]:
y_pred_con_ad[:] = [x for i,x in enumerate(y_pred_con_ad) if i not in out_Ad]

In [102]:
len(y_pred_con_ad)

1400

In [103]:
y_ts_ad=list(y_ts)

In [104]:
y_ts_ad[:] = [x for i,x in enumerate(y_ts_ad) if i not in out_Ad]

In [105]:
len(y_ts_ad)

1400

In [106]:
Q2_TS = round(r2_score(y_ts_ad, y_pred_con_ad), 2)
Q2_TS

0.54

In [107]:
RMSE_TS=round(np.sqrt(mean_absolute_error(y_ts_ad, y_pred_con_ad)), 2)
RMSE_TS

0.65